# Real-time Steps

In real-time mode, the FPGA lets Xylo run freely and only performs readout after receiving a timestep done interrupt.
This means sending input to Xylo from the FPGA is not supported in real-time mode.  Instead, the switches on the PCB must be configured such that Xylo receives input from the microphones on the PCB.


Open Dev Kit

* Call open_device() on a discovered device with Samna.

In [1]:
import warnings
warnings.filterwarnings("ignore")

import rockpool
print(f'Rockpool version {rockpool.__version__}')
import samna
print(f'Samna version {samna.__version__}')

from rockpool.devices.xylo.syns65302 import config_from_specification, mapper, XyloMonitor, XyloSamna

from rockpool.devices.xylo import find_xylo_hdks

from rockpool.transform.quantize_methods import channel_quantize
from rockpool.nn.modules import LIF, Linear
from rockpool.nn.combinators import Sequential

import numpy as np

# find_xylo_hdks() directly calls the Samna device finder and opens all available devices.
# When Samna opens a device, it will perform actions on the board. Such as, powering on the chip, resetting it,
# applying a default configuration, and configuring the FPGA for default operation.

xylo_hdks, xylo_support_modules, xylo_versions = find_xylo_hdks()
print(xylo_hdks, xylo_support_modules, xylo_versions)

Rockpool version 2.7.dev
Samna version 0.37.10.26+g11d2fc36e.dirty
from samna.xyloA3.configuration import InputInterfaceConfig
The connected Xylo HDK contains a Xylo A3. Importing `rockpool.devices.xylo.syns65302`
[<samna.xyloAudio3Boards.XyloAudio3TestBoard object at 0x73c89f93f9f0>] [<module 'rockpool.devices.xylo.syns65302' from '/home/vleite/Software/rockpool/rockpool/devices/xylo/syns65302/__init__.py'>] ['syns65302']


## Build a model and map it to hardware

In [2]:
# simple model to test

net = Sequential(
    Linear((16, 63)),
    LIF((63, 63)),

    Linear((63, 32)),
    LIF(32),
)

print(net)

# net[0].weight *= 0.05
spec = mapper(net.as_graph())
Q_spec = spec
Q_spec.update(channel_quantize(**Q_spec))
config, is_valid, msg = config_from_specification(**Q_spec)

if not is_valid:
    print(msg)

ModSequential  with shape (16, 32) {
    Linear '0_Linear' with shape (16, 63)
    LIF '1_LIF' with shape (63, 63)
    Linear '2_Linear' with shape (63, 32)
    LIF '3_LIF' with shape (32, 32)
}


## Set up Samna graphs and configuration

* Create a buffer sink for events from the chip model source node. Readout events will be captured here.
* Set Xylo configuration as desired for PDM and network config
* Set Xylo TR_WRAP register for desired time step length with default 25 MHz clock
* Set Xylo to real-time mode
* Enable PCB switches for PDM microphones
* Set FPGA module to real-time mode
   * io.write_config(0x12, 0) and io.write_config(0x31, 2)

In [3]:
# create xylo monitor
dt = 0.01
xylo_node = xylo_hdks[0]
print(xylo_node)


# PDM
config.digital_frontend.mode = samna.xyloAudio3.DigitalFrontendMode.Pdm

# TR_WRAP register
config.time_resolution_wrap = int(0x7_9FF3)

# real-time mode
config.operation_mode = samna.xyloAudio3.OperationMode.RealTime

config.debug.debug_status_update_enable = 1
config.debug.debug_input_enable = 1
config.debug.monitor_signals_enable = 1

# PCB switches
# This is physically checked on the board

# inside XyloMonitor the buffers are created and FPGA modules are set
xylo_monitor = XyloMonitor(device=xylo_node, 
    config=config, 
    dt = dt,
    output_mode='Spike')

print(xylo_monitor)

print(xylo_monitor.config)

# Create a XyloSamna structure for evolving
modSamna = XyloSamna(xylo_node, config, dt = dt)

XyloMonitor  with shape (16, 63, 32)
xyloAudio3::configuration::XyloConfiguration(operation_mode=OperationMode.RealTime, bias_enable=1, time_resolution_wrap=499699, input=xyloAudio3::configuration::InputConfig(weight_bit_shift=0, weights={ 117 -16 -65 -72 63 -27 57 -64 99 -28 72 8 11 -118 -25 -8 11 -38 -9 37 102 44 41 -48 -17 112 -39 22 65 -51 15 -97 45 -75 109 -54 -117 80 55 -115 -80 117 96 21 71 98 103 12 -127 40 -67 79 127 28 114 -20 -11 110 77 66 37 -72 -127 -105 -29 94 3 -72 -52 1 -70 68 93 -1 -76 127 49 -127 76 -99 53 -10 -41 85 -103 15 -104 -39 67 63 60 -30 110 -45 -20 -9 -80 16 -75 10 95 -89 64 127 -115 79 -62 95 110 -127 -29 -57 123 29 -126 -16 42 -62 4 48 -11 37 -43 -113 123 112 119 81 -95 107 99 -98 21 -103 -97 -117 75 -73 -77 -84 -88 42 108 12 91 -58 -32 111 -11 102 -24 1 -107 14 -33 19 56 -111 -16 91 -43 96 -43 -127 119 118 -99 -6 93 116 -6 -24 -119 20 25 -31 -43 18 -46 -127 -75 -83 -14 -33 119 66 34 -17 14 8 -6 -120 86 21 24 -118 -97 -117 109 73 12 -21 71 24 3 124 20 -96 

## Start processing and collect events

* Send TriggerProcessing event from Samna with the desired time step
* Collect readout events in the BufferSink attached to the model source node

In [4]:
# The notebook calls XyloMonitor.evolve() with some input data, which the evolve function will try to send. 
# This is not allowed in real-time mode.

import numpy as np
N = 50
a = 2
b = float

# in rd there is a AFE related record ('spike_in' or similar).
# It should be empty in silence and non empty when we make noise
out, _, rd = modSamna.evolve(input = np.zeros((N,2)), record = True)


# out, _, rec = xylo_monitor.evolve(input_data=np.ones((N,2)))
print(out)
print(rd)


  0%|          | 0/50 [00:00<?, ?it/s]

read events
[xyloAudio3::event::ReadMemoryValue(address=4608), xyloAudio3::event::ReadMemoryValue(address=4609), xyloAudio3::event::ReadMemoryValue(address=4610), xyloAudio3::event::ReadMemoryValue(address=4611), xyloAudio3::event::ReadMemoryValue(address=4612), xyloAudio3::event::ReadMemoryValue(address=4613), xyloAudio3::event::ReadMemoryValue(address=4614), xyloAudio3::event::ReadMemoryValue(address=4615), xyloAudio3::event::ReadMemoryValue(address=4616), xyloAudio3::event::ReadMemoryValue(address=4617), xyloAudio3::event::ReadMemoryValue(address=4618), xyloAudio3::event::ReadMemoryValue(address=4619), xyloAudio3::event::ReadMemoryValue(address=4620), xyloAudio3::event::ReadMemoryValue(address=4621), xyloAudio3::event::ReadMemoryValue(address=4622), xyloAudio3::event::ReadMemoryValue(address=4623), xyloAudio3::event::ReadMemoryValue(address=4624), xyloAudio3::event::ReadMemoryValue(address=4625), xyloAudio3::event::ReadMemoryValue(address=4626), xyloAudio3::event::ReadMemoryValue(ad